<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Making-predictions-with-logistic-regression" data-toc-modified-id="Making-predictions-with-logistic-regression-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Making predictions with logistic regression</a></span><ul class="toc-item"><li><span><a href="#Creating-a-query" data-toc-modified-id="Creating-a-query-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Creating a query</a></span></li><li><span><a href="#Reading-the-data-into-a-Pandas-dataframe" data-toc-modified-id="Reading-the-data-into-a-Pandas-dataframe-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Reading the data into a Pandas dataframe</a></span></li><li><span><a href="#Analyze-extracted-features" data-toc-modified-id="Analyze-extracted-features-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Analyze extracted features</a></span><ul class="toc-item"><li><span><a href="#Preprocessing-numerical-columns" data-toc-modified-id="Preprocessing-numerical-columns-1.3.1"><span class="toc-item-num">1.3.1&nbsp;&nbsp;</span>Preprocessing numerical columns</a></span></li><li><span><a href="#Preprocessing-categorical-columns" data-toc-modified-id="Preprocessing-categorical-columns-1.3.2"><span class="toc-item-num">1.3.2&nbsp;&nbsp;</span>Preprocessing categorical columns</a></span></li></ul></li></ul></li></ul></div>

# Making predictions with logistic regression

In [ ]:
import pymysql
from sqlalchemy import create_engine
import pandas as pd
import getpass    # get the password without it being visible
password = getpass.getpass()

In [ ]:

connection_string = 'mysql+pymysql://root:' + password + '@localhost/sakila' 
engine = create_engine(connection_string)

## Creating a query

I created a query which joins tables film,language,rental,payment,category and inventory. The latest is used kind of bridge table. I am interested with the columns name from category table,title, rental_rate, rating and length from rental table. Moreover, by using other joined tables, I created four other columns that could be relevant, first of them is named as noofrenting which means how many times that film is rented so far; second of them is called as last_rental which collects the last rental_date by title; third gives the average rental cost and the last one is average rental cost. 

In [ ]:
query = """ 
select ca.name as category,title,rental_duration,rating, length,count(distinct rental_id) as noofrenting, 
max(rental_date) as last_rental,
round(avg(timestampdiff(hour,rental_date,return_date))) as avg_rental_hour, 
round(avg(p.amount),2) as avg_rent_cost
from film 
join inventory using(film_id)
join rental using(inventory_id)
join payment p using(rental_id)
join film_category  using(film_id)
join category ca using(category_id)
group by title, category, rating,length, rental_duration;
"""

## Reading the data into a Pandas dataframe

In this section, the above query is read by using Pandas to obtain a dataframe, then try to understand its nature.

In [ ]:
data = pd.read_sql_query(query,engine)

In [ ]:
data.head()

In [ ]:
data.isna().sum()   # We don't have any nulls. 

In [ ]:
data.shape

In [ ]:
data.info()

## Analyze extracted features

### Preprocessing numerical columns

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [ ]:
corr_matrix = data.corr()

In [ ]:
fig, ax = plt.subplots(figsize=(10,8))
sns.heatmap(corr_matrix,
           cmap='seismic',
           center=0,
           annot=True,
           );

In [ ]:
X_num = data.select_dtypes(include = np.number)

**Before any preprocessing**

In [ ]:
from pandas.plotting import scatter_matrix
scatter_matrix(X_num,alpha=0.2,figsize=(6,6),diagonal='kde');   # not that bad, some of them seems normally distributed

**Normalizer() is applied**

In [ ]:
from sklearn.preprocessing import Normalizer
transformer = Normalizer().fit(X_num)
X_normalized= transformer.transform(X_num)

In [ ]:
X_normalized

In [ ]:
X_norm=pd.DataFrame(X_normalized, columns = X_num.columns)
scatter_matrix(X_norm,alpha=0.2,figsize=(12,12),diagonal='kde');

**Here, StandardScaler() is applied. Then I decided to go with the result of Normalizer(). Because with StandardSacaler, some of them is better, for example, noofrenting and avg_rental_hour, however others are really worse than Normalized() versions.** 

In [ ]:
from sklearn.preprocessing import StandardScaler
transformer = StandardScaler().fit(X_num)
X_standardised= transformer.transform(X_num)

In [ ]:
X_std=pd.DataFrame(X_standardised, columns= X_num.columns)
scatter_matrix(X_std,alpha=0.2,figsize=(6,6),diagonal='kde');

### Preprocessing categorical columns

In [ ]:
X_cat = data.select_dtypes(include = np.object)

In [ ]:
X_cat.head()

In [ ]:
X_cat = X_cat.drop(['title'],axis=1)   # title seems as a categorical column but actually it is irrelevant.

In [ ]:
X_cat = pd.get_dummies(X_cat,
               drop_first=True,
               columns=X_cat.columns)
X_cat

In [1]:
query_last_month = """
select title,
sum(case when rental_date like '2006-02%' then 1 else 0 end) as last_month_activity from film 
join inventory using(film_id)
join rental using(inventory_id)
group by title;
"""

In [2]:
data = pd.read_sql_query(query_last_month,engine)

NameError: name 'pd' is not defined